# Week 3. Day 1. Exercises from Chapter 7 of FSStDS. 
## Fundamentals of Social Data Science. MT 2022

Within your NEW study pod discuss the following questions. Please submit an individual assignment by 12:30pm Tuesday, October 25, 2022 on Canvas. 

# Exercise 1. How busy is twitter now? 

Using the `counts` endpoint, compare the chatter around three British politicians, Penny Mordunt, Rishi Sunak, and Boris Johnson. Use the name of the politicians in separate queries:
1. Which politician had the most mentions overall?
2. For each politician, what was the hour of pean mentions? Was it the same for each person? 

In [9]:
# Answer 1 below here
import requests
import json

def read_json(filepath: str) -> dict:
    with open(filepath, 'r') as f:
        return json.load(f)

POLITICIAN_DICT = {
    "Boris Johnson": "@BorisJohnson",
    "Penny Mordaunt": "@PennyMordaunt",
    "Rishi Sunak": "@RishiSunak",
}

count_url = "https://api.twitter.com/2/tweets/counts/recent"

# authentication
config = read_json("../twitter_config.json")
headers = {"Authorization": f"Bearer {config['bearer_token']}"}

def create_politician_query(pol_name: str) -> str:
    return f"{POLITICIAN_DICT[pol_name]} {pol_name}"

def get_tweet_count(pol_name: str) -> int:
    query = create_politician_query(pol_name)
    params = {"query": query}
    response = requests.get(count_url, headers=headers, params=params)
    return response.json()





# Answer 1 above here

In [11]:
pol_results = {pol_name: get_tweet_count(pol_name) for pol_name in POLITICIAN_DICT}

In [12]:
pol_results

{'Boris Johnson': {'data': [{'end': '2022-10-17T14:00:00.000Z',
    'start': '2022-10-17T13:46:09.000Z',
    'tweet_count': 6},
   {'end': '2022-10-17T15:00:00.000Z',
    'start': '2022-10-17T14:00:00.000Z',
    'tweet_count': 23},
   {'end': '2022-10-17T16:00:00.000Z',
    'start': '2022-10-17T15:00:00.000Z',
    'tweet_count': 17},
   {'end': '2022-10-17T17:00:00.000Z',
    'start': '2022-10-17T16:00:00.000Z',
    'tweet_count': 21},
   {'end': '2022-10-17T18:00:00.000Z',
    'start': '2022-10-17T17:00:00.000Z',
    'tweet_count': 48},
   {'end': '2022-10-17T19:00:00.000Z',
    'start': '2022-10-17T18:00:00.000Z',
    'tweet_count': 28},
   {'end': '2022-10-17T20:00:00.000Z',
    'start': '2022-10-17T19:00:00.000Z',
    'tweet_count': 19},
   {'end': '2022-10-17T21:00:00.000Z',
    'start': '2022-10-17T20:00:00.000Z',
    'tweet_count': 14},
   {'end': '2022-10-17T22:00:00.000Z',
    'start': '2022-10-17T21:00:00.000Z',
    'tweet_count': 13},
   {'end': '2022-10-17T23:00:00.000Z',
 

# Exercise 2. Tweets across the world

Select a query to download 100 tweets. Include the locations entities. Can you find a query where no single country makes up more than 50% of the locations? How can you automate this using a bank of queries? 

In [2]:
# Answer 2 below here 



# Answer 2 above here

# Exercise 3. Comparing `praw` and `psaw`

Reddit mods have considerable power over their subreddits. In /r/Science they are notoriously aggressive in their deletion of non-scientific comments. 

PushShift is an archive of reddit data. You can access this archive using `psaw` which is otherwise very similar to `praw`. Go to /r/science and select a story that over two days old. Compare the comments that you receive from `praw` and `psaw`. Do they have similar: 
- counts of comments?
- counts of comments marked deleted?
- upvote scores on the comments? 

Posit what might account for the difference. Reflect on how this could intervene in making claims about activity on Reddit.

In [5]:
# Exercise 3 code part below here




# Exercise 3 code part above here

## Exercise 3 reflections below here 



## Exercise 3 reflections above here

# Exercise 4 - Comparing comments 
(in case PushShift isn't working for you)

People can submit the same link to multiple subreddits. You can find out where else a link was submitted on Reddit by xx. 

Select a story that has been submitted to at least two subreddits. Compare the comments for each of these:
- Which subreddit has more comments? Would this have been unexpected? Why?
- Are there any overlap in the users who comment on these stories across the multiple subreddits?
- Summarise the scores of the comments versus the story (`data.ups`). Is the ratio of the top comment score to the ups the same? 
- Which story has more comments that have a score below zero? As a percentage? 


In [8]:
# Exercise 4 below here



# Exercise 4 above here